<a href="https://colab.research.google.com/github/sumnayak22/Sentiment_Analysis/blob/main/Sentiment_Analysis_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workflow tweet_data --> Data PROcessing --> Train Test split--> ML Model (LR)
# New Data --> Trained Model --> Positvive or negative Prediction


In [1]:
!pip install kaggle

In [2]:
#Configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp /kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat '/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Import Twitter sentiment data set


In [4]:
#API to fix the datset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
#Extracting the compressed dataset
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the Dependencies

In [6]:
import numpy as np
import pandas as pd # for structured tables
import re           # Regular expression --> for pattern matching or research through the data
from nltk.corpus import stopwords   #
from nltk.stem.porter import PorterStemmer # reduce a word to it's root word
from sklearn.feature_extraction.text import TfidfVectorizer  # Actual data to numerical data
from sklearn.model_selection import train_test_split    # for traing and evaluating model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# Printing Stopwords in english
print(stopwords.words('english'))
# these words dosent have any influencial meaning to data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA Processing

In [9]:
# Loading the data from csv file to pandas dataframe
twitter_data =pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding ='ISO-8859-1')

In [10]:
# for checking the no. of rows and columns
twitter_data.shape

(1599999, 6)

In [11]:
# Printing first 5 rows of dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
# Naming the columns
column_names= ['target', 'id', 'date', 'flag' , 'user' , 'text' ]
twitter_data =pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding ='ISO-8859-1')

In [13]:
# for checking the no. of rows and columns
twitter_data.shape

(1600000, 6)

In [14]:
# Printing first 5 rows of dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
# Counting a number of missing values
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [16]:
# Checking the distribution of target column
twitter_data['target'].value_counts()
# this is even distribution so ML Algo run properly

,count
target,
0,800000
4,800000


Convert the target 4 to 1


In [17]:
twitter_data.replace({'target' : {4:1}}, inplace =True)
#inplace = True for replicating the changes into original dataset

In [18]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 ---> Negative tweet
1 ---> Positive tweet

**Stemming**

Reduce a word to its root word

Example : actor, actress, acting = act

In [19]:
# Content is each tweet will be processed
port_stem= PorterStemmer()
def stemming (content):
  stemmed_content= re.sub('[^a-zA-Z]',' ', content) # remove all the letters that are not alphabet
  stemmed_content= stemmed_content.lower()          # convert all the letters to lowercase
  stemmed_content= stemmed_content.split()          # splits all the words and makes a list
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
                                                    # convert to root words and we are not doing this if word is stopwords
  stemmed_content= ' '.join(stemmed_content)        #Combine the word together

  return stemmed_content

Apply the above function to twitter_data

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [68]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [69]:
twitter_data['stemmed_content']

,stemmed_content
0,switchfoot http twitpic com zl awww bummer sho...
1,upset updat facebook text might cri result sch...
2,kenichan dive mani time ball manag save rest g...
3,whole bodi feel itchi like fire
4,nationwideclass behav mad see
...,...
1599995,woke school best feel ever
1599996,thewdb com cool hear old walt interview http b...
1599997,readi mojo makeov ask detail
1599998,happi th birthday boo alll time tupac amaru sh...


In [70]:
twitter_data['target']

,target
0,0
1,0
2,0
3,0
4,0
...,...
1599995,1
1599996,1
1599997,1
1599998,1


Separating Data and Label

In [71]:
X= twitter_data['stemmed_content'].values
Y= twitter_data['target'].values

Spilitting the data into traing_data and testing_data

In [72]:
X_train,X_test,Y_train, Y_test= train_test_split(X,Y,test_size=0.2, stratify =Y, random_state=2)
#0.2 means will 20% go to my test data and 80% will got training data
#stratify =Y, I want equal distribution of my 0 and 1 in my traing data
# random_state=2, ..splitting will be the same every time you run the code...
#can be 5 6 or any other number....This means that the selection of samples for training and testing will be different.

In [73]:
X.shape,X_train.shape, X_test.shape

((1600000,), (1280000,), (320000,))

In [74]:
original_X_train = [X_train]  # Your original training data (raw text)
original_X_test = [X_test]    # Your original test data (raw text)

**Featuring Extraction**

Vectorization

Convert test to numerical

In [75]:
Vectorizer= TfidfVectorizer()

X_train=Vectorizer.fit_transform(X_train)
X_test=Vectorizer.transform(X_test)

In [76]:

print(X_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [77]:
print(X_test)

  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538
  (319995, 107868)	0.33399349737546963
  (319995, 109379)	0.3020896484890833
  (319995, 155493)	0.2770682832971669
  (319995, 2133

**Traning the ML Model**

In [78]:
model =LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

**Model Evalution**

In [79]:
# Accuracy Score on the training data
X_train_prediction= model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train,X_train_prediction)

In [80]:
print('Accuracy score on the training data : ', training_data_accuracy)

Accuracy score on the training data :  0.79871953125


In [81]:
# Accuracy Score on the test data
X_test_prediction= model.predict(X_test)
testing_data_accuracy= accuracy_score(Y_test,X_test_prediction)

In [82]:
print('Accuracy score on the testing data : ', training_data_accuracy)

Accuracy score on the testing data :  0.79871953125


Model Accuracy =79.87%

**Saving the trained model**

In [83]:
import pickle

In [84]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))
#wb writing

Using a saved model or future Prediction

In [87]:
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))
# rb is reading

In [89]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0) :
  print('Negative Tweet')
else :
  print('Positive Tweet')

1
[1]
Positive Tweet


In [91]:
X_new = X_test[500]
print(Y_test[500])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0) :
  print('Negative Tweet')
else :
  print('Positive Tweet')

1
[1]
Positive Tweet
